In [90]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
from client import Client
from network import Net2nn
from supported_modles import Supported_modles
from sklearn.linear_model import (
    LogisticRegression,
    SGDClassifier,
)
from sklearn.model_selection import GridSearchCV
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from numpy import mean
from numpy import std
import pandas as pd

In [91]:
selected_model = Supported_modles.SGD_classifier
client1 = Client("node1","0.0.0.0", 5001, selected_model)
dataset = client1.load_data('data/Wednesday-workingHours.pcap_ISCX.csv', True)

Creating node1.


In [92]:
client1.preprocess_data(dataset, True)
client1.prep_data()

439683
251723


In [93]:
X_train = client1.x
y_train = client1.y

In [94]:
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# create model
model = SGDClassifier(
                loss="log",
                alpha=0.01,
                penalty='l1',
                max_iter=50
            )
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='f1_macro', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.960 (0.008)


In [104]:
dataset = client1.load_data("data/UNSW_NB15_train-set.csv")
test_dataset = client1.load_data("data/UNSW_NB15_test-set.csv")
df = pd.concat([test_dataset,dataset], ignore_index=True)

client1.preprocess_data(df, False)
client1.prep_data()

In [105]:
X_train = client1.x
y_train = client1.y

In [114]:
cv = KFold(n_splits=10, random_state=None, shuffle=False)
# create model
model = SGDClassifier(
                loss="log",
                alpha=0.0001,
                penalty='l1',
                max_iter=10
            )
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='f1_micro', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

/home/tk/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/tk/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/tk/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/tk/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
/home/tk/anaconda3/lib/python3.9/site-packag

Accuracy: 0.891 (0.134)


/home/tk/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:696: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


In [115]:
scores

array([0.93573424, 0.99906861, 0.9975163 , 0.80898789, 0.94621236,
       0.40181621, 0.77421608, 0.98408879, 0.99091897, 0.76071096,
       0.85284073, 0.89886681, 0.84344924, 0.90250718, 0.8857409 ,
       0.93456493, 0.98346658, 0.95870527, 0.98765815, 0.97259955])

In [56]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
import numpy as np
from sklearn.datasets import make_classification
from torch import nn

In [58]:
X = X_train.astype(np.float32)
y = y_train.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.fc1=nn.Linear(78,256)
        self.fc2=nn.Linear(256,64)
        self.fc3=nn.Linear(64,2)
        self.relu = nn.ReLU()
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x

In [85]:
cv = KFold(n_splits=5, random_state=None, shuffle=False)
# create model
net = NeuralNetClassifier(
    MyModule,
    max_epochs=30,
    criterion=nn.CrossEntropyLoss(),
    optimizer=torch.optim.SGD,
    lr=0.001,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='f1_micro', cv=cv, n_jobs=-1)
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.884 (0.080)
